# 第10章 深層学習によって実現できる画像処理・言語処理を知ろう(1-4節)
ここでは、深層学習を使った物体検出アルゴリズムについて学んでいきます。

Google Colaboratory上で実行する場合、ランタイムがGPUになっていることを確認して下さい

In [ ]:
#Colaboratory環境の設定
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MathProgramming/Chapter10

In [ ]:
#ライブラリの設定
!pip install -q -r ./requirements1.txt

## 10-3 YOLOを用いて物体検出を行ってみよう

In [ ]:
#データセットのダウンロード及び解凍を行います。
#ダウンロード済みでない場合以下を実行して下さい。
!wget http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
!tar -xvf ./VOCtrainval_06-Nov-2007.tar

In [ ]:
#yolov3-tf2のダウンロード
!git clone https://github.com/zzh8829/yolov3-tf2.git ./yolov3_tf2
%cd ./yolov3_tf2
!git checkout c43df87d8582699aea8e9768b4ebe8d7fe1c6b4c
%cd ../

In [ ]:
#YOLOの学習済みモデルのダウンロード
!wget https://pjreddie.com/media/files/yolov3-tiny.weights 

In [ ]:
#ダウンロードしたYOLOの学習済みモデルをKerasから利用出来る形に変換
!python ./yolov3_tf2/convert.py --weights ./yolov3-tiny.weights --output  ./yolov3_tf2/checkpoints/yolov3-tiny.tf --tiny

In [ ]:
from PIL import Image

#ダウンロードしたデータセットの画像の内１枚を表示
Image.open("./VOCdevkit/VOC2007/JPEGImages/006626.jpg")

In [ ]:
#表示した画像のアノテーションデータの表示
annotation = open("./VOCdevkit/VOC2007/Annotations/006626.xml").read()
print(annotation)

In [ ]:
import xmltodict
import numpy as np
from tensorflow.keras.utils import Sequence
import math
import yolov3_tf2.yolov3_tf2.dataset as dataset

yolo_max_boxes = 100

#アノテーションデータの変換
def parse_annotation(annotation, class_map):
    label = []
    width = int(annotation['size']['width'])
    height = int(annotation['size']['height'])
    
    if 'object' in annotation:
        if type(annotation['object']) != list:
            tmp = [annotation['object']]
        else:
            tmp = annotation['object']
            
        for obj in tmp:
            _tmp = []
            _tmp.append(float(obj['bndbox']['xmin']) / width)
            _tmp.append(float(obj['bndbox']['ymin']) / height)
            _tmp.append(float(obj['bndbox']['xmax']) / width)
            _tmp.append(float(obj['bndbox']['ymax']) / height)
            _tmp.append(class_map[obj['name']])
            label.append(_tmp)

    for _ in range(yolo_max_boxes - len(label)):
      label.append([0,0,0,0,0])
    return label

In [ ]:
from yolov3_tf2.yolov3_tf2.dataset import transform_images

#学習時に画像データを必要な分だけ読み込むためのクラス
class ImageDataSequence(Sequence):
    def __init__(self, file_name_list, batch_size,  anchors, anchor_masks, class_names, data_shape=(256,256,3)):
        
        #クラス名とそれに対応する数値、という形の辞書を作る
        self.class_map = {name: idx for idx, name in enumerate(class_names)}
        self.file_name_list = file_name_list

        self.image_file_name_list = ["./VOCdevkit/VOC2007/JPEGImages/"+image_path + ".jpg" for image_path in self.file_name_list]
        self.annotation_file_name_list = ['./VOCdevkit/VOC2007/Annotations/' + image_path+ ".xml" for image_path in self.file_name_list]

        self.length = len(self.file_name_list)
        self.data_shape = data_shape
        self.batch_size = batch_size
        self.anchors = anchors
        self.anchor_masks = anchor_masks

        self.labels_cache = [None for i in range(self.__len__())]

    #１バッチごとに自動的に呼ばれる。画像データとそのラベルを必要な分だけ読み込んで返す
    def __getitem__(self, idx):
        images = []
        labels = []
        
        #現在のバッチが何回目か、がidx変数に入っているため、それに対応するデータを読み込む
        for index in range(idx*self.batch_size, (idx+1)*self.batch_size):

          #アノテーションデータをラベルとして使える形に変換する
          annotation = xmltodict.parse((open(self.annotation_file_name_list[index]).read()))
          label = parse_annotation(annotation["annotation"], self.class_map)
          labels.append(label)

          #画像データの読み込みと加工
          img_raw = tf.image.decode_jpeg(open(self.image_file_name_list[index], 'rb').read(), channels=3)
          img = transform_images(img_raw, self.data_shape[0])
          images.append(img)
        
        #ラベルに対しても前処理をするが、時間がかかるため１度読み込んだらキャッシュとして保存する
        if self.labels_cache[idx] is None:
          labels = tf.convert_to_tensor(labels, tf.float32)
          labels = dataset.transform_targets(labels, self.anchors, self.anchor_masks, self.data_shape[0])
          self.labels_cache[idx] = labels
        else: 
          labels = self.labels_cache[idx]

        images = np.array(images)
        return images, labels

    def __len__(self):
        return math.floor(len(self.file_name_list) / self.batch_size)


In [ ]:
from  yolov3_tf2.yolov3_tf2.models import  YoloV3Tiny, YoloLoss
from yolov3_tf2.yolov3_tf2.utils import freeze_all
import tensorflow as tf

batch_size=16
data_shape=(416,416,3)
class_names =  ["person", "bird", "cat","cow","dog", "horse","sheep", "aeroplane", "bicycle", "boat", "bus", "car", "motorbike", "train", "bottle", "chair", "diningtable", "pottedplant", "sofa", "tvmonitor"]

anchors = np.array([(10, 14), (23, 27), (37, 58),
                              (81, 82), (135, 169),  (344, 319)],
                             np.float32) / data_shape[0]
anchor_masks = np.array([[3, 4, 5], [0, 1, 2]])

# yolov3_tf2で定義されているtiny YOLOのモデルを読み込む
model_pretrained = YoloV3Tiny(data_shape[0], training=True, classes=80)
model_pretrained.load_weights("./yolov3_tf2/checkpoints/yolov3-tiny.tf").expect_partial()

model = YoloV3Tiny(data_shape[0], training=True, classes=len(class_names))
#ここで、学習済みモデルの出力層以外の重みだけを取り出す
model.get_layer('yolo_darknet').set_weights(model_pretrained.get_layer('yolo_darknet').get_weights())
#出力層以外を学習しないようにする
freeze_all(model.get_layer('yolo_darknet'))

In [ ]:
loss = [YoloLoss(anchors[mask], classes=len(class_names)) for mask in anchor_masks]
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss=loss, run_eagerly=False)

#モデルの構造を出力
model.summary()

In [ ]:
train_file_name_list = open("./VOCdevkit/VOC2007/ImageSets/Main/train.txt").read().splitlines()
validation_file_name_list = open("./VOCdevkit/VOC2007/ImageSets/Main/val.txt").read().splitlines()

train_dataset = ImageDataSequence(train_file_name_list, batch_size, anchors, anchor_masks, class_names, data_shape=data_shape)
validation_dataset = ImageDataSequence(validation_file_name_list, batch_size, anchors, anchor_masks, class_names, data_shape=data_shape)

In [ ]:
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=30)

In [ ]:
#学習した重みの保存
model.save_weights('./saved_models/model_yolo_weights')

## 10-4 物体検出を行った結果を評価してみよう

In [ ]:
from absl import app, logging, flags
from absl.flags import FLAGS
app._run_init(['yolov3'], app.parse_flags_with_usage)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from yolov3_tf2.yolov3_tf2.utils import draw_outputs

yolo_trained = YoloV3Tiny(classes=len(class_names))
#保存した重みの読み込み
yolo_trained.load_weights('./saved_models/model_yolo_weights').expect_partial()

In [ ]:
img_file_name = "./VOCdevkit/VOC2007/JPEGImages/"+"006626" + ".jpg"

#画像の読み込み
img_raw = tf.image.decode_jpeg(open(img_file_name, 'rb').read(), channels=3)
img = transform_images(img_raw, data_shape[0])
img = np.expand_dims(img, 0)


#予測開始
boxes, scores, classes, nums = yolo_trained.predict(img)

In [ ]:
img = img_raw.numpy()

#予測結果を画像に書き込み
img = draw_outputs(img, (boxes, scores, classes, nums), class_names)

#予測結果を書き込んだ画像の表示
plt.figure(figsize=(10,10))
plt.imshow(img)
plt.show()

In [ ]:
#学習済みの重みをそのまま利用する場合

FLAGS.yolo_iou_threshold = 0.5
FLAGS.yolo_score_threshold = 0.5

yolo_class_names = [c.strip() for c in open("./yolov3_tf2/data/coco.names").readlines()]

yolo = YoloV3Tiny(classes=80)
#重みの読み込み
yolo.load_weights("./yolov3_tf2/checkpoints/yolov3-tiny.tf").expect_partial()

In [ ]:
img_file_name = "./VOCdevkit/VOC2007/JPEGImages/"+"006626" + ".jpg"

img_raw = tf.image.decode_jpeg(open(img_file_name, 'rb').read(), channels=3)
img = transform_images(img_raw, data_shape[0])
img = np.expand_dims(img, 0)
#予測開始
boxes, scores, classes, nums = yolo.predict(img)

In [ ]:
img = img_raw.numpy()
img = draw_outputs(img, (boxes, scores, classes, nums), yolo_class_names)

plt.figure(figsize=(10,10))
plt.imshow(img)
plt.show()